<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_0/DDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install optuna

In [1]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [2]:
path="/content/drive/Shareddrives/Deep Learning/datasets/DDI/"

In [3]:
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding_test.csv")

In [ ]:
print(train.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

In [ ]:
print(train.label.value_counts())
print(data_sinonimi.label.value_counts())
print(data_embedding.label.value_counts())

In [4]:
with open(path+"word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Set up per allenamento Modello

In [5]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

TRAINSET

In [14]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train.label.to_numpy().reshape(-1, 1)
)

In [15]:
sentences_train = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

TESTSET

In [6]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1)
)

In [7]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI

In [12]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1)
)

In [13]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING

In [10]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1)
)

In [11]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [17]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Il massimo è 92


Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [ ]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [18]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [20]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [21]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Carica optuna results e inizializza il modello, oppure salva il modello oppure carica solo i pesi del modello 

In [19]:
best_params = optuna.load_study(study_name="DDI", storage="sqlite:///"+path+"ddi2013-type/optuna_ddi_studio_0.db").best_params

In [ ]:
print(f'{best_params}')

{'batch_size': 89, 'dropout': 0.63, 'units': 81}


In [20]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
#train the model
result = model.fit(embedded_trainset,
                   five_hot_train,
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

In [ ]:
#save the model
model.save_weights(path+'ddi2013-type/DDI_w_studio_0.h5')

In [21]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'ddi2013-type/DDI_w_studio_0.h5')

EVALUATE ALL DATASET

In [ ]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

211/211 [==============================] - 8s 27ms/step - loss: 0.0788 - accuracy: 0.9739 </br>
DATASET ORIGINARIO[0.0771881714463234, 0.975557804107666]

In [22]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 11s 156ms/step - loss: 0.7563 - accuracy: 0.7763
DATASET TEST[0.7961077094078064, 0.7689636945724487]


In [35]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 11s 150ms/step - loss: 1.0687 - accuracy: 0.7950
DATASET SINONIMI[1.0179492235183716, 0.8043742179870605]


In [36]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET EMBEDDING{result_base}')

65/65 [==============================] - 9s 136ms/step - loss: 1.0235 - accuracy: 0.7735
DATASET EMBEDDING[1.023476004600525, 0.7734768390655518]
